In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import load_workbook
from openpyxl.styles import numbers

In [2]:
# IMPORTAR EL EXCEL

ruta = "C:/Users/9 ----- SIG/Downloads/sni.xlsx"
df = pd.read_excel(ruta, dtype=str)

In [3]:
# CONVERTIR COLUMNAS A OBJETOS

# Prepara columna DOCUMENTO vacía como str
df['DOCUMENTO'] = df['DOCUMENTO'].astype(str)

## SABER NÚMERO DE DOCUMENTO

In [4]:
# 2) SCRAPING CON SELENIUM
driver = webdriver.Chrome()
driver.get("https://hecaa.mineducacion.gov.co/hecaa-snies/content/admin/login/login.jsf")
time.sleep(0.5)
driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-user']").send_keys("1826-admin")
driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-password']").send_keys("1826Uan9123")
driver.find_element(By.ID, "loginForm:submitLogin").click()

# Navegar al módulo
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_85"))).click()
time.sleep(0.3)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_88"))).click()
time.sleep(0.3)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_89"))).click()
time.sleep(0.3)

for i in range(len(df)):
    participante = str(df.at[i, "SNIES"])  # ya es str, pero por si acaso
    campo = driver.find_element(By.ID, "fDatos:idPart")
    campo.clear()
    campo.send_keys(participante)
    WebDriverWait(driver, 4).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='Buscar']"))
    ).click()
    time.sleep(0.1)

    tds = driver.find_elements(By.TAG_NAME, "td")
    if len(tds) >= 13:
        tipo_doc = tds[11].text.strip()
        num_doc  = tds[12].get_attribute("textContent")  # conserva espacios
        df.at[i, "TIPO"]     = tipo_doc
        df.at[i, "DOCUMENTO"] = str(num_doc)             # convierto a str
    else:
        df.at[i, "TIPO"]     = ""
        df.at[i, "DOCUMENTO"] = ""

driver.quit()

# 3) ABRIR PLANTILLA OFICIAL Y RELLENAR
ruta_plantilla = "C:/Users/9 ----- SIG/Downloads/espacios.xlsx"
wb = load_workbook(ruta_plantilla)
ws = wb.active

for i in range(len(df)):
    fila = i + 2
    doc_ant   = str(df.at[i, "DOCUMENTO"])
    tipo_ant  = str(df.at[i, "TIPO"]).strip()
    doc_nuevo = doc_ant.replace(" ", "")

    ws.cell(row=fila, column=1).value = tipo_ant
    ws.cell(row=fila, column=2).value = doc_ant
    ws.cell(row=fila, column=3).value = tipo_ant
    ws.cell(row=fila, column=4).value = doc_nuevo
    ws.cell(row=fila, column=5).value = 2

    # Forzar formato TEXTO en columnas B y D
    ws.cell(row=fila, column=2).number_format = numbers.FORMAT_TEXT
    ws.cell(row=fila, column=4).number_format = numbers.FORMAT_TEXT

C:\Users\9 ----- SIG\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
# 4) GUARDAR
salida = "C:/Users/9 ----- SIG/Downloads/final.xlsx"
wb.save(salida)
print("✅ Plantilla SNIES lista:", salida)

## SACAR MAS INFO DEL PARITICIPANTE

In [8]:
# IMPORTAR EL EXCEL

ruta = "C:/Users/9 ----- SIG/Downloads/sni.xlsx"
df = pd.read_excel(ruta)
df

# CONVERTIR COLUMNAS A OBJETOS

df = df.astype({
    "TIPO": "object",
    "DOCUMENTO": "str",
    "1er_NOMBRE": "object",
    "2do_NOMBRE": "object",
    "1er_APELLIDO": "object",
    "2do_APELLIDO": "object",
    "SEXO": "object",
    "FECHA_NA": "object"
})

# Crear el navegador
driver = webdriver.Chrome()
# Abrir la página de inicio de sesión
driver.get("https://hecaa.mineducacion.gov.co/hecaa-snies/content/admin/login/login.jsf")
time.sleep(0.5)

# Ingresar usuario y contraseña
usuario = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-user']")
usuario.send_keys("1826-admin")
contrasena = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-password']")
contrasena.send_keys("1826Uan9123")

# Clic en el botón de ingresar
boton_ingresar = driver.find_element(By.ID, "loginForm:submitLogin")
boton_ingresar.click()

# Ir a la sección MDM -> Novedad Poblacional
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_85"))).click()
time.sleep(0.3)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_88"))).click()
time.sleep(0.3)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_89"))).click()
time.sleep(0.3)

try: 
    # Procesar cada registro del DataFrame
    for i in range(len(df)):
        participante = str(df['SNIES'].iloc[i])
    
        input_par = driver.find_element(By.ID, "fDatos:idPart")
        input_par.clear()
        input_par.send_keys(participante)
        time.sleep(0.1)
        boton_buscar = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Buscar']")))
        boton_buscar.click()
        time.sleep(0.1)
    
        try:
            # Buscar todos los td de la página
            tds = driver.find_elements(By.TAG_NAME, "td")
        
            if len(tds) >= 13:  # Verificar que hay suficientes elementos (ahora necesitamos hasta TD 12)
                # Extraer los datos según la posición en el debug
                snies = tds[0].text.strip()  # 'SNIES-10955217'
                primer_nombre = tds[1].text.strip()  # 'CAMILO'
                segundo_nombre = tds[2].text.strip()  # 'ANDRES'
                primer_apellido = tds[3].text.strip()  # 'VELASQUEZ'
                segundo_apellido = tds[4].text.strip()  # 'RAMIREZ'
                fecha_nacimiento = tds[5].text.strip()  # '05/04/2000'
                sexo = tds[9].text.strip()  # 'Masculino'
                tipo_documento = tds[11].text.strip()  # 'CC'
                numero_documento = tds[12].get_attribute("textContent") # '1006179751'
                
                # Asignar al DataFrame en el orden solicitado
                df.at[i, "TIPO"] = tipo_documento
                df.at[i, "DOCUMENTO"] = f'"{numero_documento}"'
                df.at[i, "1er_NOMBRE"] = primer_nombre
                df.at[i, "2do_NOMBRE"] = segundo_nombre
                df.at[i, "1er_APELLIDO"] = primer_apellido
                df.at[i, "2do_APELLIDO"] = segundo_apellido
                df.at[i, "SEXO"] = sexo
                df.at[i, "FECHA_NA"] = fecha_nacimiento
                
                print(f"✅ Datos extraídos para {tipo_documento} {numero_documento}: {primer_nombre} {segundo_nombre} {primer_apellido} {segundo_apellido}")
                
            else:
                # Si no hay suficientes elementos, llenar con valores por defecto
                df.at[i, "TIPO_DOCUMENTO"] = "0"
                df.at[i, "DOCUMENTO_NUMERO"] = "0"
                df.at[i, "1er_NOMBRE"] = "0"
                df.at[i, "2do_NOMBRE"] = "0"
                df.at[i, "1er_APELLIDO"] = "0"
                df.at[i, "2do_APELLIDO"] = "0"
                df.at[i, "SEXO"] = "0"
                df.at[i, "FECHA_NACIMIENTO"] = "0"
                print(f"⚠️ No se encontraron suficientes datos para el registro {i}")
            
        except Exception as e:
            # Si hay error, llenar con valores por defecto
            df.at[i, "TIPO_DOCUMENTO"] = "0"
            df.at[i, "DOCUMENTO_NUMERO"] = "0"
            df.at[i, "1er_NOMBRE"] = "0"
            df.at[i, "2do_NOMBRE"] = "0"
            df.at[i, "1er_APELLIDO"] = "0"
            df.at[i, "2do_APELLIDO"] = "0"
            df.at[i, "SEXO"] = "0"
            df.at[i, "FECHA_NACIMIENTO"] = "0"
            print(f"⚠️ Error al extraer datos para el registro {i}: {e}")
except Exception as e:
    print(f"❌ Error general en la ejecución: {e}")
finally:
    driver.quit()
    print("✅ Navegador cerrado. Proceso terminado.")

# GUARDAR INFORMACIÓN EN UN EXCEL

ruta = "C:/Users/9 ----- SIG/Downloads/snidatos2.xlsx"
df.to_excel(ruta, index=False)
print("✅ Archivo guardado en descargas")

✅ Datos extraídos para TI 90090153519 : MARIA CAMILA TORRES CASTRO
✅ Navegador cerrado. Proceso terminado.
✅ Archivo guardado en descargas
